# Travelling Salesman Problem (TSP) using genetic algorithms

In [92]:
import random
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import imageio.v2 as imageio
import geopandas as gpd
import contextily as ctx

## French towns initialisation (population choosen)

In [93]:
towns = [
    ("Paris", 48.8566, 2.3522),
    ("Lyon", 45.7640, 4.8357),
    ("Marseille", 43.2965, 5.3698),
    ("Lille", 50.6292, 3.0573),
    ("Bordeaux", 44.8378, -0.5792),
    ("Toulouse", 43.6047, 1.4442),
    ("Nice", 43.7102, 7.2620),
    ("Nantes", 47.2184, -1.5536),
    ("Strasbourg", 48.5734, 7.7521),
    ("Montpellier", 43.6119, 3.8772),
    ("Rennes", 48.1173, -1.6778),
    ("Le Havre", 49.4944, 0.1079),
    ("Dijon", 47.3220, 5.0415),
    ("Brest", 48.3904, -4.4861),
    ("Limoges", 45.8336, 1.2611),
    ("Perpignan", 42.6887, 2.8948),
    ("Metz", 49.1193, 6.1757),
    ("Orléans", 47.9029, 1.9093),
    ("Mulhouse", 47.7508, 7.3359)
]

## Initial population initialization function

`create_initial_population` function performs the following steps:
- Initializes an empty list to hold the population of routes.
- Loops population_size times to create the specified number of routes.
- In each iteration, it creates a copy of the towns list.
- Shuffles the order of towns in the copied list to create a unique route.
- Adds the shuffled route to the population list.
- Returns the list of generated routes.

In [94]:
def create_initial_population(towns, population_size):
    population = []
    for _ in range(population_size):
        route = towns[:]
        random.shuffle(route)
        population.append(route)
    return population

## GeoPy distance calculation

`geopy_distance` function performs the following steps:
- Uses the geopy library to calculate the geodesic distance between two coordinate points.
- Returns the distance in kilometers.

`total_route_distance` function performs the following steps:
- Iterates through a route (a list of cities with their coordinates).
- Calculates the total distance by summing the distances between each pair of consecutive cities and the distance from the last city back to the first city.
- Returns the total distance of the route.

In [95]:
def geopy_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

def total_route_distance(route):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += geopy_distance(route[i][1:], route[i+1][1:])
    total_distance += geopy_distance(route[-1][1:], route[0][1:])  # Return to starting point
    return total_distance

## Fitness evaluation function

`fitness` function performs the following steps:
- It calculates the total distance of the given route using total_route_distance(route).
- It returns the inverse of the total distance, which means shorter routes have higher fitness values and longer routes have lower fitness values.

In [96]:
def fitness(route):
    return 1 / total_route_distance(route)

## Crossover function (using Order Crossover (OX))

`crossover` function performs the following steps:
- Randomly selects two crossover points in the parent routes.
- Initializes an empty child route with None values.
- Copies a segment from parent1 to the child route based on the selected crossover points.
- Creates a list of remaining values from parent2 that are not already in the child.
- Fills in the unoccupied positions in the child route with the remaining values from parent2.
- Returns the newly created child route.

In [97]:
def crossover(parent1, parent2):
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = [None] * len(parent1)
    child[start:end] = parent1[start:end]
    fill_values = [item for item in parent2 if item not in child]
    fill_index = 0
    for i in range(len(child)):
        if child[i] is None:
            child[i] = fill_values[fill_index]
            fill_index += 1
    return child

## Mutation function (using Swap mutation)

`mutate` function performs the following steps:
- Iterates over each city in the route.
- For each city, determines whether to apply a mutation based on the mutation_rate.
- If a mutation is applied, swaps the current city with another randomly chosen city in the route.
- Returns the route, which may now include one or more mutations.

In [98]:
def mutate(route, mutation_rate=0.01):
    for i in range(len(route)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(route) - 1)
            route[i], route[j] = route[j], route[i]
    return route

## Selection function (using Tournament selection)

`tournament_selection` function performs the following steps:
- Randomly selects a subset of individuals from the population (the tournament).
- Sorts these individuals based on their fitness values in descending order.
- Returns the individual with the highest fitness value from the selected subset.

In [99]:
def tournament_selection(population, fitnesses, tournament_size=5):
    selected = random.sample(list(zip(population, fitnesses)), tournament_size)
    selected = sorted(selected, key=lambda x: x[1], reverse=True)
    return selected[0][0]

## Current generation plot function

### Using Natural Earth data

Future warning on (download from Natural Earth Data site) :
- [Natural Earth Data](https://www.naturalearthdata.com/downloads/110m-cultural-vectors/)
- world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
- france = world[world.name == 'France']

In [100]:
def plot_route_ned(route, generation, best_distance):
    fig, ax = plt.subplots(figsize=(10, 8))

    # Load world map data and focus on France
    world = gpd.read_file('NE Admin Countries/ne_110m_admin_0_countries.shp')
    france = world[world.NAME == 'France']
    france.boundary.plot(ax=ax)

    # Plot towns
    lats, lons = zip(*[(town[1], town[2]) for town in route])
    ax.plot(lons, lats, 'ro--', markersize=5)

    start_town = route[0]
    end_town = route[-1]
    ax.plot(start_town[2], start_town[1], 'go', markersize=10, label='Start')
    ax.plot(end_town[2], end_town[1], 'bo', markersize=10, label='End')

    for town in route:
        ax.text(town[2], town[1], town[0], fontsize=8)

    ax.set_title(f'Generation {generation} - Best Distance: {best_distance:.2f} km')
    ax.set_xlim(-5, 10)
    ax.set_ylim(41, 52)
    plt.savefig(f'pictures/generation_{generation}.png')
    plt.close()

### Using OpenStreetMap

In [101]:
def plot_route_gpd(route, generation, best_distance):
    fig, ax = plt.subplots(figsize=(10, 8))

    # Convert town coordinates to a GeoDataFrame
    gdf = gpd.GeoDataFrame(route, columns=['Name', 'Latitude', 'Longitude'])
    gdf['geometry'] = gpd.points_from_xy(gdf['Longitude'], gdf['Latitude'])
    gdf.set_crs(epsg=4326, inplace=True)

    # Plot the route on the map
    gdf.plot(ax=ax, color='red', markersize=50, zorder=5)
    for i, town in gdf.iterrows():
        ax.text(town['Longitude'], town['Latitude'], town['Name'], fontsize=12, ha='right')

    # Plot lines between towns
    for i in range(len(route) - 1):
        ax.plot([route[i][2], route[i+1][2]], [route[i][1], route[i+1][1]], 'ro-')

    # Plot the line returning to the start town
    #ax.plot([route[-1][2], route[0][2]], [route[-1][1], route[0][1]], 'ro-')

    # Highlight the start and end towns
    start_town = gdf.iloc[0]
    end_town = gdf.iloc[-1]
    ax.plot(start_town.geometry.x, start_town.geometry.y, 'go', markersize=10, label='Start')
    ax.plot(end_town.geometry.x, end_town.geometry.y, 'bo', markersize=10, label='End')

    # Add a basemap
    ctx.add_basemap(ax, crs=gdf.crs, source=ctx.providers.OpenStreetMap.Mapnik)

    ax.set_title(f'Generation {generation} - Best Distance: {best_distance:.2f} km')
    ax.set_xlim(-5, 10)
    ax.set_ylim(41, 52)
    plt.legend()
    plt.savefig(f'pictures/generation_{generation}.png')
    plt.close()

## Genetic algorithm main function with animation

In [102]:
def genetic_algorithm(towns, population_size, generations, mutation_rate=0.01, map="ned"):
    population = create_initial_population(towns, population_size)
    best_route = None
    best_distance = float('inf')
    images = []

    for generation in range(generations):
        fitnesses = [fitness(route) for route in population]
        
        new_population = []
        for _ in range(population_size):
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate)
            new_population.append(child)
        
        population = new_population
        for route in population:
            current_distance = total_route_distance(route)
            if current_distance < best_distance:
                best_distance = current_distance
                best_route = route

        if map == "ned":
            plot_route_ned(best_route, generation, best_distance)
        else:
            plot_route_gpd(best_route, generation, best_distance)
        
        images.append(imageio.imread(f'pictures/generation_{generation}.png'))

    imageio.mimsave(f'tsp_evolution_{map}.gif', images, duration=0.5)
    return best_route, best_distance

### Parameters

In [103]:
population_size = len(towns)
generations = 500
mutation_rate = 0.04
map = "gpd"
# map = "ned"

### Genetic algorithm launch

In [104]:
best_route, best_distance = genetic_algorithm(towns, population_size, generations, mutation_rate, map)

print("Best Route:", [town[0] for town in best_route])
print("Best Distance:", best_distance, "km")

/var/folders/y0/8vkbb8t13x706lcddt3g102w0000gn/T/ipykernel_8739/1710579535.py:6: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf['geometry'] = gpd.points_from_xy(gdf['Longitude'], gdf['Latitude'])
/var/folders/y0/8vkbb8t13x706lcddt3g102w0000gn/T/ipykernel_8739/1710579535.py:6: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=G

Best Route: ['Lille', 'Paris', 'Le Havre', 'Rennes', 'Brest', 'Nantes', 'Bordeaux', 'Toulouse', 'Perpignan', 'Montpellier', 'Marseille', 'Nice', 'Lyon', 'Limoges', 'Orléans', 'Dijon', 'Mulhouse', 'Strasbourg', 'Metz']
Best Distance: 3850.3846569071256 km
